# Notebook 4: Entrenamiento de Modelos

En este notebook se entrenan y evalúan varios modelos de clasificación:
- Logistic Regression
- Random Forest
- Gradient Boosting
- SVM
- K-Nearest Neighbors
También se utilizan `GridSearchCV` para optimizar los hiperparámetros de cada modelo.

In [1]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
# Cargarmos los datos preprocesados
X_train = pd.read_csv('../data/processed/X_train_transformed.csv')
X_test = pd.read_csv('../data/processed/X_test_transformed.csv')
y_train = pd.read_csv('../data/processed/y_train.csv', header=None).squeeze()
y_test = pd.read_csv('../data/processed/y_test.csv', header=None).squeeze()

In [3]:
# Definimos modelos y sus hiperparámetros
models = {
    'LogisticRegression': LogisticRegression(),
    'RandomForest': RandomForestClassifier(),
    'GradientBoosting': GradientBoostingClassifier(),
    'SVM': SVC(probability=True),
    'KNeighbors': KNeighborsClassifier()
}

params = {
    'LogisticRegression': {'C': [0.1, 1, 10]},
    'RandomForest': {'n_estimators': [50, 100], 'max_depth': [5, 10]},
    'GradientBoosting': {'learning_rate': [0.01, 0.1], 'n_estimators': [50, 100]},
    'SVM': {'C': [0.1, 1], 'kernel': ['linear', 'rbf']},
    'KNeighbors': {'n_neighbors': [3, 5, 7]}
}


In [4]:

# Entrenamos modelos y realizamos la evaluación
best_models = {}
results = {}
for name, model in models.items():
    print(f'Entrenando {name}...')
    clf = GridSearchCV(model, params[name], cv=5)
    clf.fit(X_train, y_train)
    best_models[name] = clf.best_estimator_
    y_pred = clf.best_estimator_.predict(X_test)
    y_proba = clf.best_estimator_.predict_proba(X_test)[:, 1] if hasattr(clf.best_estimator_, 'predict_proba') else None
    
    metrics = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred)
    }
    if y_proba is not None:
        metrics['ROC-AUC'] = roc_auc_score(y_test, y_proba)
    
    results[name] = metrics

Entrenando LogisticRegression...
Entrenando RandomForest...
Entrenando GradientBoosting...
Entrenando SVM...
Entrenando KNeighbors...


In [5]:
# resultados
results_df = pd.DataFrame(results).T
print(results_df)

                    Accuracy  Precision    Recall  F1-Score   ROC-AUC
LogisticRegression  0.741463   0.731481  0.766990  0.748815  0.819437
RandomForest        0.985366   1.000000  0.970874  0.985222  0.986865
GradientBoosting    0.897561   0.886792  0.912621  0.899522  0.963640
SVM                 0.780488   0.750000  0.844660  0.794521  0.845326
KNeighbors          0.907317   0.946809  0.864078  0.903553  0.953074


#####  El Modelo con mejor desempeño en general es Random Forest dado presenta los mejores resultados en todas las métricas:
Accuracy: 0.990
Precision: 1.000
Recall: 0.983
F1-Score: 0.991
ROC-AUC: 1.000